In [40]:
# Standard Imports
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle
import eli5

# Transformers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

# Modeling Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, confusion_matrix, classification_report
from IPython.display import display, Markdown

# Pipelines
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as PipeImb
from imblearn.pipeline import make_pipeline


# Machine Learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTENC
from sklearn.linear_model import LogisticRegression


from xgboost import XGBClassifier


In [97]:
#Look at pipeline and what it does
# from sklearn import set_config
#set_config(display='diagram')
#display(search2)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('smote',
                                        SMOTENC(categorical_features=age                          True
businesstravel               True
department                   True
distancefromhome             True
education                   False
educationfield               True
environmentsatisfaction     False
gender                       True
jobinvolvement              False
joblevel                    False
jobrole                      True
jobsatisfaction             False
maritalstatus                True
monthlyinco...
                                                                          'percentsalaryhike',
                                                                          'totalworkingyears',
                                                                          'yearsatcompany',
                                                                          'yearsincurrentrole',
                                                                          'yearssincelastpromotion',
                                                                          'yearswithcurrmanager'])])),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__max_depth': [6, 25, 50, 70],
                         'classifier__min_samples_leaf': [1, 2, 10],
                         'classifier__min_samples_split': [2, 5, 10],
                         'classifier__n_estimators': [100, 300, 500]},
             scoring='recall')

# User defined functions

In [41]:
def a_xy_split(dataframe,y_name):
   
    #Get xy split and train/test data ready
    X = dataframe.drop([y_name], axis=1)
    y = dataframe.attrition
    y = LabelEncoder().fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    #Prep numerical and other column types for use in preprocessor
    numerical=X.select_dtypes(include='number')
    categorical=X.select_dtypes(include='object')
    numerical_columns=list(numerical.columns)
    categorical_columns=list(categorical.columns)
    columns=list(X.columns)
    cat_col_bool=[True if x in categorical_columns else False for x in columns]

    return X_train, X_test, y_train, y_test,numerical_columns,categorical_columns,cat_col_bool

In [65]:
def b_preprocessors():
    #Creating ss transformer to scale the continuous numerical data with StandardScaler()
    ss = Pipeline(steps=[('ss', StandardScaler())])
    #--------------------------------------------------------------------
    #Creating ohe transformer to encode the categorical data with OneHotEncoder()
    ohe = Pipeline(steps=[('ohe', OneHotEncoder(drop='first'))])
    #--------------------------------------------------------------------
    #Creating preprocess column transformer to combine the ss and ohe pipelines
    preprocess = ColumnTransformer(
                        transformers=[
                            ('num', ss, numerical_columns),
                            ('cat', ohe, categorical_columns)
                        ])
    return preprocess



In [43]:
def c_get_standard_params():
        #Creating parameter grid for Random Forest
        rand_forest_parms = {'classifier__n_estimators': [100, 300, 500],
                'classifier__max_depth':[6, 25, 50, 70],
                'classifier__min_samples_split': [2, 5, 10],
                'classifier__min_samples_leaf': [1, 2, 10]}
        xgb_params = { 'classifier__max_depth': [3,6,10],
                'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
                'classifier__n_estimators': [100, 500, 1000],
                'classifier__colsample_bytree': [0.3, 0.7],
                'classifier__subsample':[.6,1]}

        log_regress_params={'classifier__solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                        'classifier__penalty':['none','l1','l2','elasticnet'],
                        'classifier__C':[100,10,1.0, 0.1,0.01]}
        knn_params={'classifier__n_neighbors':[1, 5, 10],
        'classifier__weights': ['uniform', 'distance'],
        'classifier__metric': ['euclidean', 'manhattan', 'minkowski']}

        return rand_forest_parms,xgb_params,log_regress_params,knn_params

In [84]:
def _evaluation(y, y_hat, title = 'Confusion Matrix'):
    cm = confusion_matrix(y, y_hat)
    precision = precision_score(y, y_hat)
    recall = recall_score(y, y_hat)
    accuracy = accuracy_score(y,y_hat)
    f1 = f1_score(y,y_hat)
    print('Accuracy: ', accuracy)
    print('Recall: ', recall)
    print('Precision: ', precision)
    print('F1: ', f1)
    sns.heatmap(cm,  cmap= 'PuBu', annot=True, fmt='g', annot_kws=    {'size':20})
    plt.xlabel('predicted', fontsize=18)
    plt.ylabel('actual', fontsize=18)
    plt.title(title, fontsize=18)
    
    plt.show()

In [105]:
def d_cross_validate_grid(preprocessor, classifier, smote_type='NC', param_grid="none", cv="none", scoring="recall", conf_matrix=False, grid_search=False):
    cv = cv

    #Set correct pipeline for use in the formula
    pipeline = PipeImb(steps=[
    ('preprocess', preprocessor),
    ('classifier', classifier)])


    if smote_type.lower()=="nc":
        pipeline.steps.insert(0,('smote',SMOTENC(categorical_features=X_train.dtypes==object,random_state=1234,n_jobs=-1)))


    if grid_search==True:
        search = GridSearchCV(pipeline, param_grid=param_grid, cv=cv,scoring=scoring)
    else:
        search=pipeline
# Run train and validation methods, print average result for each
    train_acc = []
    train_rec=[]
    train_prec=[]
    test_acc = []
    test_rec=[]
    test_prec=[]
    for train_ind, val_ind in cv.split(X_train, y_train):
        X_t, y_t = X_train.iloc[train_ind], y_train[train_ind]
        search.fit(X_t, y_t)
        y_hat_t = search.predict(X_t)
        train_acc.append(accuracy_score(y_t, y_hat_t))
        train_rec.append(recall_score(y_t,y_hat_t))
        train_prec.append(precision_score(y_t,y_hat_t))
        X_val, y_val = X_train.iloc[val_ind], y_train[val_ind]
        y_hat_val = search.predict(X_val)
        test_acc.append(accuracy_score(y_val, y_hat_val))
        test_rec.append(recall_score(y_val, y_hat_val))
        test_prec.append(precision_score(y_val, y_hat_val))

    if conf_matrix==True:
        print("Validation set confusion matrix")
        print(_evaluation(y_val, y_hat_val))
        print()
    print('Avg Training Accuracy: {}'.format(np.mean(train_acc)))
    print("Avg Training Recall: {}".format(np.mean(train_rec)))
    print("Avg Training Precision: {}".format(np.mean(train_prec)))
    print()
    print('Avg Validation Accuracy: {}'.format(np.mean(test_acc)))
    print("Avg Validation Recall: {}".format(np.mean(test_rec)))
    print("Avg Validation Precision: {}".format(np.mean(test_prec)))
    print()

# If grid search used, print best estimator's features
    if (grid_search==True):
        ohe_cols = list(search.best_estimator_.named_steps['preprocess'].named_transformers_['cat'].named_steps['ohe'].get_feature_names(
        input_features=categorical_columns))
        num_feats = list(numerical_columns)
        num_feats.extend(ohe_cols)
        feat_imp = eli5.explain_weights_df(search.best_estimator_.named_steps['classifier'], top=10, feature_names=num_feats)
        print(feat_imp)
        print()
        print('Grid Search Best Params:')
        print()
        print(search.best_params_)


In [86]:
def e_validate_using_test(preprocessor, classifier, grid, cv):
    final_pipeline = GridSearchCV(PipeImb(steps=[
            ('preprocess', preprocessor),
            ('classifier', classifier)
        ]), grid, cv=cv)
    #Fit and predict on train data
    final_pipeline.fit(X_train, y_train)

    train_pred = final_pipeline.best_estimator_.predict(X_train)
    #train_pred=final_pipeline.predict(X_train)

    print('Evaluation on training data \n')
    print(_evaluation(y_train, train_pred))
    print('\n')
    #Predict on test data

    test_pred = final_pipeline.best_estimator_.predict(X_test)
    #test_pred = final_pipeline.predict(X_test)
    print('Evaluation on testing data \n')
    print(_evaluation(y_test, test_pred))

# Code here

In [100]:
df=pd.read_csv(r"C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\Ironhack_final_project\Data\4a.IBM_data_cleaned_dropped_categorized - Copy.csv")
y_name='attrition'
X_train, X_test, y_train, y_test,numerical_columns,categorical_columns,cat_col_bool=a_xy_split(df,y_name)
preprocess=b_preprocessors()
rand_forest_parms,xgb_params,log_regress_params,knn_params=c_get_standard_params()

In [106]:
d_cross_validate_grid(preprocess,LogisticRegression(),param_grid=log_regress_params,cv=KFold(),grid_search=True)

c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio par